In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

campaign_data = pd.read_csv("C:/Users/sudhe/Desktop/Internship-FUTURENSE/US_Pathway/cl_campaign.csv")

# Preprocessing: Handle missing values
campaign_data.fillna(0, inplace=True)
campaign_data['platform'] = campaign_data['platform'].astype(str)
campaign_data['adset_name'] = campaign_data['adset_name'].astype(str)

# Features and target variable
X = campaign_data[['impressions', 'clicks', 'click_through_rate', 'leads', 'platform', 'adset_name']]
y = campaign_data['total_spent']

# Preprocessing and modeling pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), ['impressions', 'clicks', 'click_through_rate', 'leads']),
        ('cat', OneHotEncoder(), ['platform', 'adset_name'])
    ])),
    ('regressor', RandomForestRegressor())
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
pipeline.fit(X_train, y_train)

def get_user_input():
    platforms = input('Enter target platforms (comma-separated, e.g., Google,Facebook,LinkedIn): ').split(',')
    
    input_data = {}
    for platform in platforms:
        platform = platform.strip()
        impressions = int(input(f'Enter target impressions for {platform}: '))
        clicks = int(input(f'Enter target clicks for {platform}: '))
        ctr = float(input(f'Enter target click-through rate (CTR) for {platform}: '))
        leads = int(input(f'Enter target leads for {platform}: '))
        adset_name = input(f'Enter new adset name for {platform}: ')
        
        input_data[platform] = {
            'impressions': impressions,
            'clicks': clicks,
            'click_through_rate': ctr,
            'leads': leads,
            'adset_name': adset_name
        }
    
    return input_data

def predict_spending(pipeline, input_data):
    results = {}
    for platform, data in input_data.items():
        input_df = pd.DataFrame({
            'impressions': [data['impressions']],
            'clicks': [data['clicks']],
            'click_through_rate': [data['click_through_rate']],
            'leads': [data['leads']],
            'platform': [platform],
            'adset_name': [data['adset_name']]
        })
        predicted_spending = pipeline.predict(input_df)[0]
        results[platform] = predicted_spending
    return results

# Get user input and predict spending
input_data = get_user_input()
predicted_spending = predict_spending(pipeline, input_data)

# Output the predicted spending
for platform, spending in predicted_spending.items():
    print(f'Predicted Spending for {platform}: {spending}')


Predicted Spending for LinkedIn: 14611.374799999992
Predicted Spending for Google: 16138.692900000015


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'R² Score: {r2}')


Mean Absolute Error (MAE): 181.76577373927054
Mean Squared Error (MSE): 303107.04540574405
R² Score: 0.873800755125757


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import yfinance as yf

campaign_data = pd.read_csv("cl_campaign.csv")

campaign_data.fillna(0, inplace=True)
campaign_data['platform'] = campaign_data['platform'].astype(str)
campaign_data['adset_name'] = campaign_data['adset_name'].astype(str)

X = campaign_data[['impressions', 'clicks', 'click_through_rate', 'leads', 'platform', 'adset_name']]
y = campaign_data['total_spent']

pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), ['impressions', 'clicks', 'click_through_rate', 'leads']),
        ('cat', OneHotEncoder(), ['platform', 'adset_name'])
    ])),
    ('regressor', RandomForestRegressor())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

def get_user_input():
    platforms = input('Enter target platforms (comma-separated, e.g., Google,Facebook,LinkedIn): ').split(',')
    
    input_data = {}
    for platform in platforms:
        platform = platform.strip()
        impressions = int(input(f'Enter target impressions for {platform}: '))
        clicks = int(input(f'Enter target clicks for {platform}: '))
        ctr = float(input(f'Enter target click-through rate (CTR) for {platform}: '))
        leads = int(input(f'Enter target leads for {platform}: '))
        adset_name = input(f'Enter new adset name for {platform}: ')
        
        input_data[platform] = {
            'impressions': impressions,
            'clicks': clicks,
            'click_through_rate': ctr,
            'leads': leads,
            'adset_name': adset_name
        }
    
    return input_data

def predict_spending(pipeline, input_data):
    results = {}
    for platform, data in input_data.items():
        input_df = pd.DataFrame({
            'impressions': [data['impressions']],
            'clicks': [data['clicks']],
            'click_through_rate': [data['click_through_rate']],
            'leads': [data['leads']],
            'platform': [platform],
            'adset_name': [data['adset_name']]
        })
        predicted_spending = pipeline.predict(input_df)[0]
        results[platform] = predicted_spending
    return results

def fetch_last_market_price(stock_name):
    data = yf.Ticker(stock_name).history(period="1mo")
    last_market_price = data['Close'].iloc[-1]
    print("Last market price for", stock_name, ":", last_market_price)
    return last_market_price

def is_investment_feasible(investment_amount, last_market_price):
    if investment_amount <= last_market_price:
        print("Investment is feasible.")
    else:
        print("Investment is not feasible.")

input_data = get_user_input()
predicted_spending = predict_spending(pipeline, input_data)

for platform, spending in predicted_spending.items():
    print(f'Predicted Spending for {platform}: {spending}')

stock_name = input("Enter share name (ticker symbol, e.g., AAPL for Apple): ")
last_market_price = fetch_last_market_price(stock_name)

investment_amount = float(input("Enter investment amount: "))
is_investment_feasible(investment_amount, last_market_price)


Predicted Spending for Google: 15914.191300000011
Last market price for GOOGL : 167.0
Investment is not feasible.


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
import yfinance as yf
import numpy as np

campaign_data = pd.read_csv("C:/Users/sudhe/Desktop/Internship-FUTURENSE/US_Pathway/cl_campaign.csv")

campaign_data.fillna(0, inplace=True)
campaign_data['platform'] = campaign_data['platform'].astype(str)
campaign_data['adset_name'] = campaign_data['adset_name'].astype(str)

X = campaign_data[['impressions', 'clicks', 'click_through_rate', 'leads', 'platform', 'adset_name']]
y = campaign_data['total_spent']

pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(transformers=[
        ('num', StandardScaler(), ['impressions', 'clicks', 'click_through_rate', 'leads']),
        ('cat', OneHotEncoder(), ['platform', 'adset_name'])
    ])),
    ('regressor', RandomForestRegressor())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

def get_user_input():
    platforms = input('Enter target platforms (comma-separated, e.g., Google,Facebook,LinkedIn): ').split(',')
    
    input_data = {}
    for platform in platforms:
        platform = platform.strip()
        impressions = int(input(f'Enter target impressions for {platform}: '))
        clicks = int(input(f'Enter target clicks for {platform}: '))
        ctr = float(input(f'Enter target click-through rate (CTR) for {platform}: '))
        leads = int(input(f'Enter target leads for {platform}: '))
        adset_name = input(f'Enter new adset name for {platform}: ')
        
        input_data[platform] = {
            'impressions': impressions,
            'clicks': clicks,
            'click_through_rate': ctr,
            'leads': leads,
            'adset_name': adset_name
        }
    
    return input_data

def predict_spending(pipeline, input_data):
    results = {}
    for platform, data in input_data.items():
        input_df = pd.DataFrame({
            'impressions': [data['impressions']],
            'clicks': [data['clicks']],
            'click_through_rate': [data['click_through_rate']],
            'leads': [data['leads']],
            'platform': [platform],
            'adset_name': [data['adset_name']]
        })
        predicted_spending = pipeline.predict(input_df)[0]
        results[platform] = predicted_spending
    return results

def analyze_stock_trends(platform_ticker_map):
    trend_results = {}
    for platform, ticker in platform_ticker_map.items():
        stock_data = yf.Ticker(ticker).history(period="1mo")
        closing_prices = stock_data['Close']
        volumes = stock_data['Volume']
        
        start_price = closing_prices.iloc[0]
        end_price = closing_prices.iloc[-1]
        percentage_change = ((end_price - start_price) / start_price) * 100
        
        avg_volume = volumes.mean()
        percentage_volume_change = ((volumes.iloc[-1] - volumes.iloc[0]) / volumes.iloc[0]) * 100
        volatility = closing_prices.std()
        max_price = closing_prices.max()
        min_price = closing_prices.min()
        price_range = max_price - min_price
        moving_avg_10d = closing_prices.rolling(window=10).mean().iloc[-1]
        
        delta = closing_prices.diff().dropna()
        up, down = delta.clip(lower=0), -delta.clip(upper=0)
        roll_up, roll_down = up.rolling(window=14).mean(), down.rolling(window=14).mean()
        rs = roll_up / roll_down
        rsi = 100 - 100 / (1 + rs)
        rsi_14d = rsi.iloc[-1]
        
        trend_direction = "Upward" if percentage_change > 0 else "Downward"
        
        trend_results[platform] = {
            'start_price': start_price,
            'end_price': end_price,
            'percentage_change': percentage_change,
            'avg_volume': avg_volume,
            'percentage_volume_change': percentage_volume_change,
            'volatility': volatility,
            'max_price': max_price,
            'min_price': min_price,
            'price_range': price_range,
            'moving_avg_10d': moving_avg_10d,
            'rsi_14d': rsi_14d,
            'trend_direction': trend_direction
        }
    
    return trend_results

def evaluate_feasibility(trend_results):
    feasibility_results = {}
    for platform, trend in trend_results.items():
        is_feasible = (
            trend['percentage_change'] > 0 and
            trend['rsi_14d'] < 70 and
            trend['volatility'] < (trend['max_price'] - trend['min_price']) * 0.1
        )
        feasibility_results[platform] = "Feasible" if is_feasible else "Not Feasible"
    
    return feasibility_results

platform_ticker_map = {
    'Google': 'GOOGL',
    'Facebook': 'META',
    'LinkedIn': 'MSFT'
}

input_data = get_user_input()
predicted_spending = predict_spending(pipeline, input_data)

trend_results = analyze_stock_trends(platform_ticker_map)

feasibility_results = evaluate_feasibility(trend_results)

for platform, spending in predicted_spending.items():
    print(f'Predicted Spending for {platform}: {spending}')

print("\nStock Trend Analysis:")
for platform, trend in trend_results.items():
    print(f"\nPlatform: {platform}")
    print(f"Start price 30 days ago: {trend['start_price']}")
    print(f"Last market price: {trend['end_price']}")
    print(f"Percentage change over the last 30 days: {trend['percentage_change']:.2f}%")
    print(f"Average volume over the last 30 days: {trend['avg_volume']}")
    print(f"Percentage change in volume traded: {trend['percentage_volume_change']:.2f}%")
    print(f"Volatility (Standard deviation of closing prices): {trend['volatility']}")
    print(f"Max closing price over the last 30 days: {trend['max_price']}")
    print(f"Min closing price over the last 30 days: {trend['min_price']}")
    print(f"Price range (Max - Min): {trend['price_range']}")
    print(f"10-day moving average: {trend['moving_avg_10d']}")
    print(f"RSI (14-day): {trend['rsi_14d']:.2f}")
    print(f"Overall trend direction: {trend['trend_direction']}")
    print(f"Investment Feasibility: {feasibility_results[platform]}")


Predicted Spending for Google: 16369.621700000012
Predicted Spending for Facebook: 13690.005299999997

Stock Trend Analysis:

Platform: Google
Start price 30 days ago: 185.41000366210938
Last market price: 167.0
Percentage change over the last 30 days: -9.93%
Average volume over the last 30 days: 24243614.285714287
Percentage change in volume traded: 119.12%
Volatility (Standard deviation of closing prices): 6.730463141270847
Max closing price over the last 30 days: 191.17999267578125
Min closing price over the last 30 days: 167.0
Price range (Max - Min): 24.17999267578125
10-day moving average: 177.71900024414063
RSI (14-day): 20.71
Overall trend direction: Downward
Investment Feasibility: Not Feasible

Platform: Facebook
Start price 30 days ago: 519.5599975585938
Last market price: 465.70001220703125
Percentage change over the last 30 days: -10.37%
Average volume over the last 30 days: 14426323.80952381
Percentage change in volume traded: 40.39%
Volatility (Standard deviation of clos